In [1]:
%cd '/Users/Simona/'

/Users/Simona


In [2]:
%matplotlib inline
import numpy as np
import torchvision
from torchvision import datasets, transforms
import torch
import torch.nn as nn
from PIL import Image
import logging
import faiss
import matplotlib.pyplot as plt
from cycler import cycler
import record_keeper
import pytorch_metric_learning
logging.getLogger().setLevel(logging.INFO)
logging.info("VERSION %s"%pytorch_metric_learning.__version__)

from record_keeper import RecordKeeper, RecordWriter

INFO:root:VERSION 0.9.87.dev4


In [3]:
import Classifier_v5
from Classifier_v5 import RF
from Classifier_v5 import LR
from Classifier_v5 import MLP

In [4]:
# Set the image transforms
# Resnets
train_transform = transforms.Compose([transforms.Resize(224),
                                    #transforms.RandomResizedCrop(scale=(0.16, 1), ratio=(0.75, 1.33), size=64),
                                    #transforms.RandomHorizontalFlip(0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_transform = transforms.Compose([transforms.Resize(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])




In [5]:
train_dataset =  datasets.ImageFolder('/Users/Simona/Fresno_Area/train', transform=train_transform)
val_dataset =  datasets.ImageFolder('/Users/Simona/Fresno_Area/val', transform=val_transform)
test_dataset =  datasets.ImageFolder('/Users/Simona/Fresno_Area/test', transform=val_transform)

# Resnet50 pretrained on Resisc45

In [6]:
X_train = np.loadtxt('/Users/Simona/Fresno_Area/X_train_resnet50resisc45')
X_test = np.loadtxt('/Users/Simona/Fresno_Area/X_test_resnet50resisc45')

In [7]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_test_minmax = min_max_scaler.transform(X_test)

In [8]:
X_train_minmax.shape

(2324, 2048)

In [9]:
#RF(X_train_minmax,X_test_minmax)
#LR(X_train_minmax,X_test_minmax)
MLP(X_train_minmax,X_test_minmax)

Mean accuracy: 0.8034
Standard deviation: 0.0250


# Resnet18 pretrained on Imagenet

In [ ]:
X_train = np.loadtxt('/Users/Simona/Fresno_Area/X_train_resnet18imagenet')
X_test = np.loadtxt('/Users/Simona/Fresno_Area/X_test_resnet18imagenet')

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_test_minmax = min_max_scaler.transform(X_test)

In [ ]:
X_train_minmax.shape

In [ ]:
RF(X_train_minmax,X_test_minmax)
LR(X_train_minmax,X_test_minmax)
MLP(X_train_minmax,X_test_minmax)

# PCA

In [18]:
# Set the image transforms
# PCA, K-means, Raw
train_transform = transforms.Compose([transforms.Resize(50)])
                                  
val_transform = transforms.Compose([transforms.Resize(50)])

In [19]:
train_dataset =  datasets.ImageFolder('/Users/Simona/Fresno_Area/train', transform=train_transform)
val_dataset =  datasets.ImageFolder('/Users/Simona/Fresno_Area/val', transform=val_transform)
test_dataset =  datasets.ImageFolder('/Users/Simona/Fresno_Area/test', transform=val_transform)

In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.9)

In [ ]:
# Embed tiles
n_tiles = len(train_dataset)
z_dim = 3*50*50
X_train = np.zeros((n_tiles, z_dim))

for idx in range(n_tiles):
    tile = train_dataset[idx][0]
    tile = np.asarray(tile)
    tile = tile/255.0
    tile_2d = tile.reshape(z_dim)
    tile = tile_2d.reshape(-1, 1)
    tile = pca.fit_transform(tile)
    tile = np.moveaxis(tile, -1, 0)
    X_train[idx,:] = tile

In [ ]:
# Embed tiles
n_tiles = len(test_dataset)
z_dim = 3*50*50
X_test = np.zeros((n_tiles, z_dim))

for idx in range(n_tiles):
    tile = test_dataset[idx][0]
    tile = np.asarray(tile)
    tile = tile/255.0
    tile_2d = tile.reshape(z_dim)
    tile = tile_2d.reshape(-1, 1)
    tile = pca.transform(tile)
    #tile = pca.fit_transform(tile)
    tile = np.moveaxis(tile, -1, 0)
    X_test[idx,:] = tile

In [21]:
X_train.dtype

dtype('float64')

In [ ]:
X_test.shape

In [ ]:
np.savetxt('/Users/Simona/Fresno_Area/X_train_PCA3',X_train)
np.savetxt('/Users/Simona/Fresno_Area/X_test_PCA3',X_test)

In [23]:
X_train = np.loadtxt('/Users/Simona/Fresno_Area/X_train_PCA3')
X_test = np.loadtxt('/Users/Simona/Fresno_Area/X_test_PCA3')

In [24]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_test_minmax = min_max_scaler.transform(X_test)

In [25]:
#RF(X_train_minmax,X_test_minmax)
#LR(X_train_minmax,X_test_minmax)
MLP(X_train_minmax,X_test_minmax)

Mean accuracy: 0.5003
Standard deviation: 0.0128


# K-means

In [8]:
X_test_kmeans = np.loadtxt('/Users/Simona/Fresno_Area/X_test_kmeans')
X_train_kmeans = np.loadtxt('/Users/Simona/Fresno_Area/X_train_kmeans')

In [11]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train_kmeans)
X_test_minmax = min_max_scaler.transform(X_test_kmeans)

In [12]:
RF(X_train_kmeans,X_test_kmeans)
LR(X_train_kmeans,X_test_kmeans)
MLP(X_train_kmeans,X_test_kmeans)

Mean accuracy: 0.6204
Standard deviation: 0.0298
Mean accuracy: 0.4991
Standard deviation: 0.0062
Mean accuracy: 0.5039
Standard deviation: 0.0149


# Raw features

In [ ]:
# Embed tiles
#Transformations without "to tensor"!!

n_tiles = len(train_dataset)
z_dim = 3*50*50
X_train = np.zeros((n_tiles, z_dim))

for idx in range(len(train_dataset)):
    tile = train_dataset[idx][0]
    tile = np.asarray(tile)
    tile = tile/255.0
    tile_2d = tile.reshape(z_dim)
    tile = tile_2d.reshape(-1, 1)
    tile = np.moveaxis(tile, -1, 0)
    X_train[idx,:] = tile

In [ ]:
n_tiles = len(test_dataset)
z_dim = 3*50*50
X_test = np.zeros((n_tiles, z_dim))

for idx in range(len(test_dataset)):
    tile = test_dataset[idx][0]
    tile = np.asarray(tile)
    tile = tile/255.0
    tile_2d = tile.reshape(z_dim)
    tile = tile_2d.reshape(-1, 1)
    tile = np.moveaxis(tile, -1, 0)
    X_test[idx,:] = tile

In [14]:
X_train.shape

(2324, 7500)

In [15]:
X_train.dtype

dtype('float64')

In [ ]:
np.savetxt('/Users/Simona/Fresno_Area/X_train_rawfloat',X_train)
np.savetxt('/Users/Simona/Fresno_Area/X_test_rawfloat',X_test)

In [13]:
X_test = np.loadtxt('/Users/Simona/Fresno_Area/X_test_rawfloat')
X_train = np.loadtxt('/Users/Simona/Fresno_Area/X_train_rawfloat')

In [16]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train_kmeans)
X_test_minmax = min_max_scaler.transform(X_test_kmeans)

In [17]:
RF(X_train_kmeans,X_test_kmeans)
LR(X_train_kmeans,X_test_kmeans)
MLP(X_train_kmeans,X_test_kmeans)

Mean accuracy: 0.6214
Standard deviation: 0.0309
Mean accuracy: 0.4998
Standard deviation: 0.0036
Mean accuracy: 0.5052
Standard deviation: 0.0165
